## 環境準備

In [0]:
import dlt
from pyspark.sql import functions as F, types as T

## Bronze Layer

In [0]:
@dlt.table()
def customers_bronze(table_properties={"quality":"bronze"}):
    return (
        spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("header", "true")
        .option("cloudFiles.inferColumnTypes", "true")
        .load("/Volumes/MLOps/raw/source_files/customers/")
    )

In [0]:
schema = spark.table("databricks_simulated_retail_customer_data.v01.sales_orders").schema

@dlt.table(table_properties={"quality":"bronze"})
def sales_orders_bronze():
    return (
        spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("header", "true")
        .option("quote", '"')         
        .option("escape", '"')        
        .option("multiLine", "true") 
        .option("mode", "PERMISSIVE")  
        .schema(schema)               
        .load("/Volumes/MLOps/raw/source_files/sales_orders/")
    )

## Silver Layer

In [0]:
# 共通の前処理
def not_null_and_not_blank(col_name: str):
    return (F.col(col_name).isNotNull()) & (F.trim(F.col(col_name)) != "")

In [0]:
@dlt.table(table_properties={"quality":"silver"})
@dlt.expect_or_fail("pk_must_exist", "customer_id IS NOT NULL AND trim(customer_id) != ''")
@dlt.expect_all_or_drop({
    "state_present": "state IS NOT NULL AND trim(state) != ''",
    "city_present": "city IS NOT NULL AND trim(city) != ''",
    "units_present": "units_purchased IS NOT NULL AND trim(units_purchased) != ''",
    "loyalty_present": "loyalty_segment IS NOT NULL AND trim(loyalty_segment) != ''"
})

def customers_silver():
    base = spark.readStream.table("customers_bronze").alias("base")

    df = (
        base.select("customer_id","state","city","units_purchased","loyalty_segment")
            .where(
                not_null_and_not_blank("customer_id") &
                not_null_and_not_blank("state") &
                not_null_and_not_blank("city") &
                not_null_and_not_blank("units_purchased") &
                not_null_and_not_blank("loyalty_segment")
            )
    )
    return df

In [0]:
@dlt.table(table_properties={"quality":"silver"})
@dlt.expect_or_fail("pk_must_exist", "order_number IS NOT NULL AND trim(order_number) != ''")
@dlt.expect_all_or_drop({
    "clicked_items_present": "clicked_items IS NOT NULL AND trim(clicked_items) != ''",
    "customer_id_present": "customer_id IS NOT NULL AND trim(customer_id) != ''",
    "number_of_line_items_present": "number_of_line_items IS NOT NULL AND trim(number_of_line_items) != ''",
    "order_datetime_present": "order_datetime IS NOT NULL AND trim(order_datetime) != ''",
    "ordered_products_present": "ordered_products IS NOT NULL AND trim(ordered_products) != ''",
    "promo_info_present": "promo_info IS NOT NULL AND trim(promo_info) != ''"
})

def sales_orders_silver():
    base = spark.readStream.table("sales_orders_bronze").alias("base")

    df = (
        base.select("clicked_items","customer_id","number_of_line_items","order_datetime","order_number","ordered_products","promo_info")
            .where(
                not_null_and_not_blank("clicked_items") &
                not_null_and_not_blank("customer_id") &
                not_null_and_not_blank("number_of_line_items") &
                not_null_and_not_blank("order_datetime") &
                not_null_and_not_blank("order_number") &
                not_null_and_not_blank("ordered_products") &
                not_null_and_not_blank("promo_info")
            )
    )
    return df

In [0]:
@dlt.table(table_properties={"quality":"silver"})
def sales_orders_customer_join_min():
    base = dlt.read_stream("sales_orders_silver").alias("base")

    so = (
        base.select("clicked_items","customer_id","number_of_line_items","order_datetime","order_number","ordered_products","promo_info")
    )

    c = dlt.read("customers_silver").alias("c")

    df = (
        so.join(c, "customer_id", "left")
          .select("clicked_items","customer_id","number_of_line_items","order_datetime","order_number","ordered_products","promo_info","state","city","units_purchased","loyalty_segment")
    )
    return df

## Gold Layer

In [0]:
ITEMS_SCHEMA = T.ArrayType(T.StructType([
    T.StructField("curr", T.StringType()),
    T.StructField("id",   T.StringType()),
    T.StructField("name", T.StringType()),
    T.StructField("price",T.StringType()),
    T.StructField("promotion_info", T.StringType()),
    T.StructField("qty",  T.StringType()),
    T.StructField("unit", T.StringType()),
]))

In [0]:
@dlt.table(table_properties={"quality":"gold"})
def als_interactions_30d():
    base = dlt.read("sales_orders_customer_join_min").alias("base")

    j = (
        base
        .withColumn(
            "ts",
            F.coalesce(
                F.to_timestamp(F.from_unixtime(F.col("order_datetime").cast("long"))),
                F.to_timestamp("order_datetime"),
                F.col("order_datetime").cast("timestamp")
            )
        )
        .withColumn("items", F.from_json("ordered_products", ITEMS_SCHEMA))
        .withColumn("p", F.explode("items"))
    )

    mx = j.agg(F.max("ts").alias("max_ts"))
    j = j.crossJoin(mx)
    w30 = F.col("ts") >= F.date_sub(F.col("max_ts"), 30)

    return (
        j.where(w30)
         .select(
             "customer_id",
             F.col("p.id").alias("product_id"),
             F.col("p.qty").cast("int").alias("qty")
         )
         .where(F.col("qty").isNotNull() & (F.col("qty") > 0))
         .groupBy("customer_id","product_id")
         .agg(F.sum("qty").alias("total_qty"))
         .withColumn("interaction_weight", F.log1p(F.col("total_qty")))
    )